In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
invited_info = pd.read_csv('invited_info_csv.txt', sep='\t')
question_info = pd.read_csv('question_info_csv.txt', sep='\t')
user_info = pd.read_csv('user_info_csv.txt', sep='\t')

In [3]:
invited_info = invited_info.drop(['Unnamed: 0'], axis=1)
question_info = question_info.drop(['Unnamed: 0'], axis=1)
user_info = user_info.drop(['Unnamed: 0'], axis=1)

In [4]:
all_info = invited_info.merge(question_info, left_on='qid', right_on='qid', how='left')

In [5]:
all_info = all_info.merge(user_info, left_on='uid', right_on='uid', how='left')

In [6]:
# all_info.head()

In [7]:
label_series = all_info['label']

In [8]:
label_array = np.array(label_series)

In [9]:
feature_dataframe = all_info.loc[:, 'qwid':'utag_142']

In [10]:
feature_name = list(feature_dataframe.columns)
# feature_name

In [11]:
feature_matrix = feature_dataframe.as_matrix()

In [12]:
# from sklearn.feature_selection import RFE
# from sklearn.linear_model import SGDRegressor
# model = SGDRegressor()
# selector = RFE(model, step=1)
# selector = selector.fit(feature_matrix, label_array)

# model.fit(feature_matrix, label_array)

In [13]:
validate_nolabel = pd.read_csv('./bytecup2016data/validate_nolabel.txt', sep=',')

In [14]:
# validate_nolabel.head()

In [15]:
valitate_set = validate_nolabel.merge(question_info, left_on='qid', right_on='qid', how='left')

In [16]:
valitate_set = valitate_set.merge(user_info, left_on='uid', right_on='uid', how='left')

In [17]:
# valitate_set.head()

In [18]:
valitate_set_matrix = valitate_set.loc[:,'qwid':'utag_142'].as_matrix()

In [19]:
valitate_set_matrix.shape

(30466, 172)

In [20]:
# predict_result = model.predict(valitate_set_matrix)
# predict_result

# validate_nolabel['label'] = predict_result + 0.2

In [21]:
# validate_nolabel.to_csv('validate_nolabel.csv', sep=',', index=False)

In [22]:
from sklearn.decomposition import IncrementalPCA
ipca = IncrementalPCA(n_components=90)
ipca.fit(feature_matrix)
feature_matrix_pca = ipca.transform(feature_matrix) 

In [23]:
# feature_matrix_pca.shape

In [24]:
ipca2 = IncrementalPCA(n_components=90)
ipca2.fit(valitate_set_matrix)
valitate_set_matrix_pca = ipca2.transform(valitate_set_matrix)

In [25]:
# valitate_set_matrix_pca.shape

In [26]:
# from sklearn.linear_model import SGDRegressor
# model = SGDRegressor()
# model.fit(feature_matrix_pca, label_array)

In [27]:
# predict_result = model.predict(valitate_set_matrix_pca)

In [28]:
# max(predict_result)

In [29]:
import xgboost as xgb

In [44]:
dtrain = xgb.DMatrix(feature_matrix[:160608], label=label_array[:160608], feature_names=feature_name)

In [45]:
dtest = xgb.DMatrix(feature_matrix[160608:], label=label_array[160608:], feature_names=feature_name)

In [32]:
# dtrain

In [53]:
param = {'bst:max_depth':7, 'bst:eta':1, 'silent':1, 'objective':'reg:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'mae'

In [54]:
num_round = 100

In [55]:
evallist  = [(dtest,'eval'), (dtrain,'train')]

In [56]:
bst = xgb.train( param.items(), dtrain, num_round, evallist)

[0]	eval-mae:0.413709	train-mae:0.390843
[1]	eval-mae:0.362414	train-mae:0.347444
[2]	eval-mae:0.326074	train-mae:0.316535
[3]	eval-mae:0.30002	train-mae:0.294365
[4]	eval-mae:0.281176	train-mae:0.278339
[5]	eval-mae:0.267716	train-mae:0.266872
[6]	eval-mae:0.257994	train-mae:0.258462
[7]	eval-mae:0.250967	train-mae:0.252513
[8]	eval-mae:0.245768	train-mae:0.248002
[9]	eval-mae:0.241946	train-mae:0.244637
[10]	eval-mae:0.239291	train-mae:0.24229
[11]	eval-mae:0.237178	train-mae:0.240305
[12]	eval-mae:0.23579	train-mae:0.239037
[13]	eval-mae:0.234527	train-mae:0.237836
[14]	eval-mae:0.233369	train-mae:0.236619
[15]	eval-mae:0.232697	train-mae:0.236003
[16]	eval-mae:0.23205	train-mae:0.235344
[17]	eval-mae:0.231276	train-mae:0.234531
[18]	eval-mae:0.230947	train-mae:0.234181
[19]	eval-mae:0.230541	train-mae:0.233682
[20]	eval-mae:0.230206	train-mae:0.233229
[21]	eval-mae:0.230035	train-mae:0.233
[22]	eval-mae:0.229848	train-mae:0.232758
[23]	eval-mae:0.229641	train-mae:0.232545
[24]	eval

In [37]:
# bst.get_fscore()

In [57]:
dvalidate = xgb.DMatrix(valitate_set_matrix, feature_names=feature_name)

In [58]:
pre = bst.predict(dvalidate)
# from graphviz import *

In [59]:
# fig1 = plt.figure(figsize=(15, 15))

In [60]:
validate_nolabel['label'] = pre

In [61]:
validate_nolabel.to_csv('validate_nolabel.csv', sep=',', index=False)

In [63]:
bst.get_fscore()

{'#ans': 200,
 '#tqans': 151,
 '#tqans/#ans': 237,
 '#upvotes': 253,
 '#upvotes/#ans': 271,
 'qcid': 356,
 'qtag_0': 4,
 'qtag_1': 11,
 'qtag_10': 9,
 'qtag_11': 1,
 'qtag_12': 7,
 'qtag_13': 14,
 'qtag_14': 3,
 'qtag_15': 7,
 'qtag_16': 9,
 'qtag_17': 2,
 'qtag_18': 9,
 'qtag_19': 11,
 'qtag_2': 19,
 'qtag_3': 28,
 'qtag_4': 7,
 'qtag_5': 18,
 'qtag_6': 9,
 'qtag_7': 10,
 'qtag_8': 10,
 'qtag_9': 15,
 'qwid': 324,
 'ucid': 844,
 'utag_0': 19,
 'utag_1': 7,
 'utag_100': 2,
 'utag_103': 14,
 'utag_104': 1,
 'utag_105': 3,
 'utag_106': 6,
 'utag_107': 5,
 'utag_108': 3,
 'utag_110': 3,
 'utag_111': 5,
 'utag_112': 2,
 'utag_115': 2,
 'utag_116': 4,
 'utag_117': 5,
 'utag_118': 1,
 'utag_119': 1,
 'utag_12': 1,
 'utag_120': 2,
 'utag_121': 2,
 'utag_122': 2,
 'utag_123': 1,
 'utag_124': 1,
 'utag_128': 3,
 'utag_13': 1,
 'utag_133': 2,
 'utag_14': 6,
 'utag_140': 1,
 'utag_15': 1,
 'utag_16': 5,
 'utag_18': 9,
 'utag_19': 9,
 'utag_2': 5,
 'utag_20': 9,
 'utag_21': 9,
 'utag_22': 7,
 'uta